In [ ]:
latest_gameweek = 20

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import gradio as gr
import plotly.graph_objects as go

# Data

In [ ]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
display(projections.head())
display(projections.shape)

In [ ]:
filepath = Path('../data/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='23-24']
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
df = fpl_df.groupby('name').last().reset_index()[['name', 'team_name', 'element_type', 'now_cost', 
                            'gameweek_minutes_ewm_20', 'points_per_game', 'total_points', 
                            'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_10', 'gameweek_xPoints_ewm_20', 'gameweek_xPoints_ewm_40']]
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
df['price'] = df['now_cost'] / 10.0
df['value'] = df['gameweek_xPoints_ewm_20'] / df['price']
df['value_points'] = np.sqrt( df['gameweek_xPoints_ewm_20'] *  df['value'])

expected_points_next_10gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+11, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_10_GW'})

expected_points_next_5gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+6, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_5_GW'})

expected_points_next_10gw
df = df.merge(expected_points_next_10gw, on='name', how='left')
df = df.merge(expected_points_next_5gw, on='name', how='left')
display(df)

In [ ]:
position_dict = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
df['position'] = df['element_type'].map(position_dict)

In [ ]:
df['name'].values[0]

# Gradio functions

In [ ]:
def plot_points_and_value(positions, teams, show_names, x_axis_feature):
    
    fig = go.Figure()

    df_out = df[df.position.isin(positions)].copy()
    if "Select All" not in teams:
        df_out = df_out[df_out.team_name.isin(teams)]

    fig.add_trace(
            go.Scatter(
                x=df_out[x_axis_feature],
                y=df_out['value'],
                mode="markers+text",
                hovertext=df_out['name'].values,
                showlegend=False,
                ),
        )

    if show_names:
        fig.update_traces(
            text = df_out['name'].values,
            textposition='top center',
            )

    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title=x_axis_feature,
        yaxis_title='value',
        #showlegend=True
    )

    return fig

# Demo

In [ ]:
position_list = list(df.position.unique())
team_name_list = ["Select All"]
team_name_list += list(np.sort(df.team_name.unique()))
x_axis_feature = ["gameweek_xPoints_ewm_20", 'expected_points_next_10_GW']

In [ ]:
demo = gr.Interface(
    plot_points_and_value,
    [
        gr.CheckboxGroup(position_list, label="POSITION"),
        gr.Dropdown(team_name_list, label="TEAM", multiselect=True),
        gr.Checkbox('Show player names'),
        gr.Dropdown(x_axis_feature, label="x-axis"),
    ],
    gr.Plot(),
).launch()

# DEV STUFF

In [ ]:
teams = ['Aston Villa']
positions = ['MID']

In [ ]:
fig = go.Figure()

df_out = df[df.position.isin(positions)].copy()
if "Select All" not in teams:
    df_out = df_out[df_out.team_name.isin(teams)]

fig.add_trace(
    go.Scatter(
        x=df_out['gameweek_xPoints_ewm_20'],
        y=df_out['value'],
        mode="markers+text",
        text = df_out['name'].values,
        hovertext=df_out['name'].values,
        showlegend=False,
        ),
)

fig.update_traces(textposition='top center')

fig.update_layout(
    #title="",
    template='plotly_dark',
    xaxis_title="gameweek_xPoints_ewm_20",
    yaxis_title='value',
    #showlegend=True
)

